In [4]:
!python3 -m pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


### Scraping NFL Weather

In [33]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import time

pd.options.display.max_columns = None

rowData = []

for season in [2022, 2023, 2024]:
    for i in range(1, 19):
    
        page = requests.get(f"https://www.nflweather.com/week/{season}/week-{i}")
    
        # scrape webpage
        soup = BeautifulSoup(page.content, 'html.parser')

        divs = soup.find_all('div', class_='game-box w-100 d-flex flex-column flex-lg-row align-items-center shadow-box rounded my-2 py-1')

        for div in divs:
            data = [season, i]
            date = div.find('div', class_='fw-bold text-wrap').getText()
            data.append(date.strip())

            awayTeam = div.find('div', class_='flex-centered flex-column me-1 ms-xxl-auto')
            data.append(awayTeam.getText().strip())

            homeTeam = div.find('div', class_='flex-centered flex-column me-xxl-auto')
            data.append(homeTeam.getText().strip())

            awayPoints = div.find('div', class_='game-points ps-1 pe-2')
            data.append(int(awayPoints.getText().strip()))

            homePoints = div.find('div', class_='game-points pe-1 ps-2')
            data.append(int(homePoints.getText().strip()))

            try:
                weather = div.find_all('div', class_='mx-2')
                for w in weather:
                    data.append(w.find('span').getText())
            
            except:
                continue

            rowData.append(data)

        time.sleep(1)


df = pd.DataFrame(rowData, columns = ['Season', 'Week_Number', 'Date', 'Away_Team', 'Home_Team', 'Away_Points', 'Home_Points', 'Temperature', 'Weather'])

df

,Season,Week_Number,Date,Away_Team,Home_Team,Away_Points,Home_Points,Temperature,Weather
0,2022,1,09/08/22 08:20 PM EDT,Bills,Rams,31,10,91 °F,Partly Cloudy
1,2022,1,09/11/22 01:00 PM EDT,Saints,Falcons,27,26,78 °F,Mostly Cloudy
2,2022,1,09/11/22 01:00 PM EDT,49ers,Bears,10,19,68 °F,Rain
3,2022,1,09/11/22 01:00 PM EDT,Steelers,Bengals,23,20,75 °F,Mostly Cloudy
4,2022,1,09/11/22 01:00 PM EDT,Eagles,Lions,38,35,74 °F,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...
810,2024,18,01/05/25 04:25 PM EST,Chargers,Raiders,34,20,57 °F,Mostly Cloudy
811,2024,18,01/05/25 04:25 PM EST,Seahawks,Rams,30,25,75 °F,Clear
812,2024,18,01/05/25 04:25 PM EST,Dolphins,Jets,20,32,31 °F,Clear
813,2024,18,01/05/25 04:25 PM EST,49ers,Cardinals,24,47,61 °F,Clear


In [35]:
df.to_csv('./output_data/nfl-weather-data.csv', index=False)

In [36]:
df.to_clipboard(excel=True)